# RNN

# Part 1:  Text Classification

### Part 1: Neural Network Method

### Pre-processing for Neural Network 

In [5]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import pandas as pd
from functools import reduce
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize
import nltk

In [6]:
data = pd.read_csv('axcs_train.csv')
data.head()

,ID,URL,Date,Title,InfoTheory,CompVis,Math,Abstract
0,cs-9301111,arxiv.org/abs/cs/9301111,1989-12-31,Nested satisfiability,0,0,0,Nested satisfiability A special case of the s...
1,cs-9301112,arxiv.org/abs/cs/9301112,1990-03-31,A note on digitized angles,0,0,0,A note on digitized angles We study the confi...
2,cs-9301113,arxiv.org/abs/cs/9301113,1991-07-31,Textbook examples of recursion,0,0,0,Textbook examples of recursion We discuss pro...
3,cs-9301114,arxiv.org/abs/cs/9301114,1991-10-31,Theory and practice,0,0,0,Theory and practice The author argues to Sili...
4,cs-9301115,arxiv.org/abs/cs/9301115,1991-11-30,Context-free multilanguages,0,0,0,Context-free multilanguages This article is a...


In [7]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

In [8]:
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}


In [9]:
def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, '')
    return x

def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict)

def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)

stop_words= set(stopwords.words('english'))

def stop_words_removal(s):
    stops_no = ' '.join(word for word in s.split() if word not in stop_words)
    return stops_no
W_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

from nltk.stem.wordnet import WordNetLemmatizer
def lem_text(text):
    return [lemmatizer.lemmatize(w) for w in W_tokenizer.tokenize(text)]
    

In [10]:
Abstract=data.Abstract

In [11]:
cleaning_data= []
for i in Abstract:
    Clean_data= clean_text(i)
    cleaning_data.append(Clean_data)

In [12]:
list_b = []
for i in cleaning_data:
    b= clean_numbers(i)
    list_b.append(b)

In [13]:
list_c =[]
for i in list_b:
    d=replace_contractions(i)
    list_c.append(d)

In [14]:
data['Abstract_clean'] = pd.DataFrame(list_c)
data['Abstract_clean']=data['Abstract_clean'].str.lower()

In [15]:
data.loc[:,"Abstract_data"] =  data.Abstract_clean.apply(lambda x:stop_words_removal(x))
data.loc[:,"Abstract_data"] = data.Abstract_data.apply(lem_text)
data.loc[:,"Abstract_data"] = data.Abstract_data.apply(lambda x :" ".join(x))
data=data.drop(['Abstract'], axis=1)
data=data.drop(['Abstract_clean'], axis=1)
data=data.rename(columns={'Abstract_data':'Abstract'})
data.to_csv("data_train_axcs.csv", index=False)

# Test_data

In [16]:
data_test= pd.read_csv("axcs_test.csv")
data_test.head()

,ID,URL,Date,Title,InfoTheory,CompVis,Math,Abstract
0,no-150100335,arxiv.org/abs/1501.00335,1/1/15,A Data Transparency Framework for Mobile Appli...,0,0,0,A Data Transparency Framework for Mobile Appl...
1,no-14024178,arxiv.org/abs/1402.4178,1/1/15,A reclaimer scheduling problem arising in coal...,0,0,0,A reclaimer scheduling problem arising in coa...
2,no-150100263,arxiv.org/abs/1501.00263,1/1/15,Communication-Efficient Distributed Optimizati...,0,0,1,Communication-Efficient Distributed Optimizat...
3,no-150100287,arxiv.org/abs/1501.00287,1/1/15,Consistent Classification Algorithms for Multi...,0,0,0,Consistent Classification Algorithms for Mult...
4,no-11070586,arxiv.org/abs/1107.0586,1/1/15,Managing key multicasting through orthogonal s...,0,0,0,Managing key multicasting through orthogonal ...


In [17]:
Abstract_test=data.Abstract

In [18]:
cleaning_data_test= []
for i in Abstract_test:
    Clean_data_test= clean_text(i)
    cleaning_data_test.append(Clean_data_test)

In [19]:
list_b_test = []
for i in cleaning_data_test:
    b_test= clean_numbers(i)
    list_b_test.append(b_test)

In [20]:
list_c_test =[]
for i in list_b_test:
    d_test=replace_contractions(i)
    list_c_test.append(d_test)

In [21]:
data_test['Abstract_clean_test'] = pd.DataFrame(list_c)
data_test['Abstract_clean_test']=data_test['Abstract_clean_test'].str.lower()
data_test.loc[:,"Abstract_data"] =  data_test.Abstract_clean_test.apply(lambda x:stop_words_removal(x))
data_test.loc[:,"Abstract_data"] = data_test.Abstract_data.apply(lem_text)
data_test.loc[:,"Abstract_data"] = data_test.Abstract_data.apply(lambda x :" ".join(x))
data_test=data_test.drop(['Abstract'], axis=1)
data_test=data_test.drop(['Abstract_clean_test'], axis=1)
data_test=data_test.rename(columns={'Abstract_data':'Abstract'})
data_test.to_csv("data_test_axcs.csv", index=False)

# RNN 

In the below code block we have set the seed so that when we run the code each time the output and the random values that we have assigned should not change.

<b>Field</b>- This defines a datatype and also the Instruction together for the convertion to Tensor, And then this takes many parameter and in that we have taken 3 i,e. <b>sequential</b>, <b>Tokenize</b> and <b>lower</b>. These are used to taken in a sequence of words and then tokenize them as we need and then convert them into lowercase.
So, here we are using Spacy as a Tokeniser.

<b>Spacy</b> - Spacy is used for tokenizing large amount of text data and also used in building natural language understanding system and also they help in pre-processing text for deep learning and neural networks.

In [0]:
import torch
from torchtext import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(sequential=True, tokenize = 'spacy', lower=True)
# tokenize = lambda x: x.split()
# TEXT = data.Field(sequential=True, tokenize = tokenize, lower=True) #in case you want to use your own tokenizer

LABEL = data.LabelField(dtype = torch.float, use_vocab=False, preprocessing=int)

## Preparing the data for COMPVIS

Here we will be using TabularDataset to read our data and the dataset we are using here is a csv format and then we are usding the specified datafields which we have previously defined.

Then for the train and test data we need to process the labels and the order is same as that of the columns and then the fields which we are not using will pass them to a tuple where the second element is None.

In [0]:
from torchtext.data import TabularDataset

## Applying NN model on CompVis column
compvis_data = [("ID", None), # we won't be needing the source and notes, so we pass in None as the field
                 ("URL", None),
                 ("Date", None),
                 ("Title", None),
                 ("InfoTheory", None),
                 ("CompVis", LABEL),
                 ('Math',None),
                 ("Abstract", TEXT)]

train_compvis, test_compvis = TabularDataset.splits(path='',
                                              train='axcs_train.csv', 
                                              test='axcs_test.csv', 
                                              format='csv',
                                              skip_header=True,
                                              fields=compvis_data)

Here Below we have printed the Training examples length and Testing examples length to check how they are splitting.

In [8]:
print(f'Number of training examples: {len(train_compvis)}')
print(f'Number of testing examples: {len(test_compvis)}')

Number of training examples: 54731
Number of testing examples: 19679


We are also checking with an example.

In [9]:
print(vars(train_compvis.examples[0]))

{'CompVis': 0, 'Abstract': [' ', 'nested', 'satisfiability', 'a', 'special', 'case', 'of', 'the', 'satisfiability', 'problem', ',', 'in', 'which', 'the', 'clauses', 'have', 'a', 'hierarchical', 'structure', ',', 'is', 'shown', 'to', 'be', 'solvable', 'in', 'linear', 'time', ',', 'assuming', 'that', 'the', 'clauses', 'have', 'been', 'represented', 'in', 'a', 'convenient', 'way', '.']}


Next we are building the Vocabulary, that is effectively used as a lookup table i,e every word in our data set has a corresonding index we do this operation since machine leaning cannot operate on strings.

So, each index is used to construct a one-hot encoding. in which it trasforms the srings into an inter value of vectors form and then in this we will have more zero's and only one 1's in the vetor which indicate according the index value.

The Dimentionality is the total number of unique words in the vocabulary.

The number of unique words in our Training set is " " and since it would take more memory space we would cut down the training dataset as the top 5000 words.

then we are replacing a special unknown token when there is need for those missing words.

Then we are only keeping the most common max_size tokens.

In [0]:
MAX_VOCAB_SIZE = 5000

TEXT.build_vocab(train_compvis, max_size = MAX_VOCAB_SIZE)
#LABEL.build_vocab(train_data)

In [11]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
#print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 5002


Printing the frequencies of most common words in the training set.

The vocab size 5002 because of the addition tokens is the `<unk>` token and the other is a `<pad>` token.

In [12]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 512010), ('of', 344238), ('.', 332875), (',', 307484), ('a', 218648), ('and', 213963), ('-', 184859), ('to', 176340), ('in', 174585), ('is', 130627), ('for', 120901), ('we', 110661), ('that', 89421), ('this', 74411), ('on', 68779), ('with', 66354), ('are', 57723), (' ', 54731), ('an', 49791), ('by', 49768)]


When we feed the sentence into our model we feed more than one at time and then it should follow a same size as that of the batch size and to ensure each sentence in the batch is the same size, any shorter than the longest within the batch are padded.

In [13]:
print(TEXT.vocab.itos[:10])
print(LABEL)

['<unk>', '<pad>', 'the', 'of', '.', ',', 'a', 'and', '-', 'to']


Here we are initializing the batch size as 16, then we are preparing the data by creating an iterators.

After creating we are iterating over the training and testing and then returning the batch of examples which are converted into tensors by using the indexes at every iterations.

Here below we are using `BucketIterator` which is a special iterator which would return a examples of batches with similar length and minimizing the amount of padding per examples.

Then we are replacing the tensors returned by the iterators on the GPU and this is handled by `torch.device` by passing this device to the iterator.

We also want to place the tensors returned by the iterator on the GPU (if you're using one). PyTorch handles this using `torch.device`, we then pass this device to the iterator.

In [0]:
BATCH_SIZE = 16

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, test_iterator = data.BucketIterator.splits(
    (train_compvis, test_compvis), 
    batch_size = BATCH_SIZE,
    device = device,
    sort_key = lambda x: len(x.Abstract),
    sort_within_batch = False)

In [15]:
batch_compvis = next(train_iterator.__iter__())
batch_compvis


[torchtext.data.batch.Batch of size 16]
	[.CompVis]:[torch.cuda.FloatTensor of size 16 (GPU 0)]
	[.Abstract]:[torch.cuda.LongTensor of size 345x16 (GPU 0)]

# Building a Model


Here in the Rnn class is a sub-class of nn module and then here we have used super function and defined init function, Forward function.

<b> Super</b> - Super function is used to make class inheritence more easy to use and manage and also helps in allowing new capabilities and functionalities for a method or a function.

<b>__init__</b> - In this Init function we are defining the three layers of the module and those three layers are as follows: -
- Embedding layer
- RNN layer
- Linear Layer

And these layer's parameters are already initialized to Random values.
<b>Embedding Layer</b> - 
- In embedding layer we are converting the sparse one hot encoded vectors into dense embedding vectors i,e. The sparse vectors contains more number of elements as zero's and the size of it would be very large so, we are transforming them into dense vector where the dimentionality is lot smaller than sparse and contains only real numbers.
- Embedding layer is a simple layer which is fully connected as well as it helps in reducing the dimentionality of the Input data so the RNN would learn easily from it.
- the words which have similar impact on the sentiment of the review are mapped close together in this dense vector space.

<b>RNN Layer</b> - 
- The RNN layer will take the dense vectors which is passed from the embedding layer in a sequence and then produces a hidden layer and then by applying Rnn again and again for the current word and the previous hidden state, we can produce the next hidden state ℎ𝑡=RNN(𝑥𝑡,ℎ𝑡−1).

<b>Linear Layer</b> - 
- Once we have our final hidden state,  ℎ𝑇 , (from feeding in the last word in the sequence,  𝑥𝑇 ) we feed it through a linear layer,  𝑓 , (also known as a fully connected layer), to receive our predicted sentiment,  𝑦̂ =𝑓(ℎ𝑇) .
- After we have the final hidden state ℎ𝑇 which we got from the last word in sequence 𝑥𝑇 then we feed it through a linear layer to receive the predicted sentiments 𝑦̂ =𝑓(ℎ𝑇) .


In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

Here below we are creating some instances for our RNN class and the input dimension here is the one-hot vector which is equal to Vocab size.

Then the embedding dimension is the size of the dense vector which is transformed from sparse vector to dense vector to reduce the size of the dataset so Rnn could learn easily and this depends on the size of the Vocabulary.

The hidden dimentions are nothing but the size hidden states and then this also is depending upon the size of the Vocabulary and also as mentioned above that is from the embedding the dense vector and also dependent in the complecxity of the task

The output dimentions are usually the number of classes. here we are providing with one dimention. which is a single scalar value.

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 150
HIDDEN_DIM = 200
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

Here we are defining a function by passing the above created model and returing the sum of all parameters of the training set.

In [18]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 820,901 trainable parameters


#  Train the Model for compvis

Here we are creating a Optimizer which is an algorithm where we update the parameters of the module.

So, here we are using <b>stochastic gradient descent (SGD)</b> as an optimizer. In this the first argument we are passing is the parameters that has to be updated and then followed by the learning rate.

In [0]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

Here in the below cell have defined a loss function and in PyTorch it is commonly called as Criterion.
This is bacause it is used as criteria for the function to carry out or to terminate.

Here we are using `Binary cross entropy with logits`.

Here we are restricting the predictions to 0 and 1 by suing some sigmoid or logit function.
Then after this we use some bond scalar to calculate the loss by using binary cross entropy and the `BCEWithLogitsLoss` criterion is used to carry out the sigmoid and the binary cross entropy steps.

In [0]:
criterion = nn.BCEWithLogitsLoss()

Using `.to`, we are placing the model and the criterion on the GPU (if we have one). 

In [0]:
model = model.to(device)
criterion = criterion.to(device)

After Calculating the loss we are writing a function to calculate the accuracy as mentioned at the begining.

- The Train function is made to iterate over all examples i,e. one batch at a time and before that we are training our model and which turns on the dropout and batch normalization.
- First we are putting zeros for all batches and then storing the gradient calculated by the Criterion by making all paramaters in the model to have a grad attribute.
- This is done because PyTorch does not have the capability to remove the gradients calculated i,e. zeros from the gradient calculations, so we are remoiving it manually.

- After the above process we are providing the batch of sentences as batch.text into the model which is done by the forward function as mention and explained above and then it squeeze is needed as the prediction re initially size [batch size, 1], and we need to remove the dimension of size 1 as PyTorch expects the predictions input to our criterion function to be of size [batch size].

- Loss and accuracy are then measured using our forecasts and the package.label labels, with the loss being summed over all the examples in the sample.

- We calculate the gradient of each parameter with loss.backward(), and then update the parameters using the gradients and optimizer algorithm with optimizer.step().

- `.item()` is used to extract a scalar from the tensor in turn which only contains a single value.

- then we are returning the loss and accuracy as an average acorss the number of iterations or steps or epoch and then we are specifying the len of an iterator as the number of batches in the iteartor.



In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch_compvis in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch_compvis.Abstract).squeeze(1)
        
        loss = criterion(predictions, batch_compvis.CompVis)
        
        acc = binary_accuracy(predictions, batch_compvis.CompVis)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Evaluate function is same as the train function but here we are not updating the paramters when evaluating.

model.eval() puts the model in "evaluation mode", this turns off dropout and batch normalization. Again, we are not using them in this model, but it is good practice to include them.

No gradients are calculated on PyTorch operations inside the with no_grad() block. This causes less memory to be used and speeds up computation.

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch_compvis.Abstract).squeeze(1)
            
            loss = criterion(predictions, batch_compvis.CompVis)
            
            acc = binary_accuracy(predictions, batch_compvis.CompVis)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

We'll also create a function to tell us how long an epoch takes to compare training times between models.

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Here we are training the model through multiple epochs and then as we train model one by one we are passing through all the examples in the training set so the RNN learns it well.

In [26]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_compvis_loss, train_compvis_acc = train(model, train_iterator, optimizer, criterion)    
    end_time = time.time()

    epoch_compvis_mins, epoch_compvis_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_compvis_mins}m {epoch_compvis_secs}s')
    print(f'\tTrain Loss: {train_compvis_loss:.3f} | Train Acc: {train_compvis_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 38s
	Train Loss: 0.183 | Train Acc: 95.86%
Epoch: 02 | Epoch Time: 0m 38s
	Train Loss: 0.172 | Train Acc: 95.89%
Epoch: 03 | Epoch Time: 0m 38s
	Train Loss: 0.171 | Train Acc: 95.90%
Epoch: 04 | Epoch Time: 0m 38s
	Train Loss: 0.171 | Train Acc: 95.90%
Epoch: 05 | Epoch Time: 0m 38s
	Train Loss: 0.171 | Train Acc: 95.89%


In [0]:
torch.save(model.state_dict(), 'RNN_model.pt')

In [28]:
model.load_state_dict(torch.load('RNN_model.pt'))

<All keys matched successfully>

In [0]:
y_compvis_predict = []
y_compvis_test = []

model.eval()
with torch.no_grad():
    for batch in test_iterator:
        predictions = model(batch.Abstract).squeeze(1)
        rounded_preds = torch.round(torch.sigmoid(predictions))
        y_compvis_predict += rounded_preds.tolist()
        y_compvis_test += batch.CompVis.tolist()
        #acc = binary_accuracy(predictions, batch.CompVis)

In [30]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, matthews_corrcoef
import numpy as np

y_compvis_predict = np.asarray(y_compvis_predict)
y_compvis_test = np.asarray(y_compvis_test)
print(confusion_matrix(y_compvis_test,y_compvis_predict))
recall_compvis=recall_score(y_compvis_test,y_compvis_predict,average='macro')
precision_compvis=precision_score(y_compvis_test,y_compvis_predict,average='macro')
f1score_compvis=f1_score(y_compvis_test,y_compvis_predict,average='macro')
accuracy_compvis=accuracy_score(y_compvis_test,y_compvis_predict)
matthews_compvis = matthews_corrcoef(y_compvis_test,y_compvis_predict) 
print('Accuracy_compvis: '+ str(accuracy_compvis))
print('Macro Precision_compivs: '+ str(precision_compvis))
print('Macro Recall_compvis: '+ str(recall_compvis))
print('Macro F1 score_compvis:'+ str(f1score_compvis))
print('MCC_compvis:'+ str(matthews_compvis))

[[17464    63]
 [ 2141    11]]
Accuracy_compvis: 0.8880024391483307
Macro Precision_compivs: 0.5197209068287875
Macro Recall_compvis: 0.5007585349465073
Macro F1 score_compvis:0.47526369343181973
MCC_compvis:0.007735372520170296


In [31]:
print("The number of true negatives for Compvis classifier of NN model is: " + str(confusion_matrix(y_compvis_test,y_compvis_predict)[0][0]))
print("The number of false positives for Compvis classifier of NN model is: " + str(confusion_matrix(y_compvis_test,y_compvis_predict)[0][1]))
print("The number of false negatives for compvis classifier of NN model is: " + str(confusion_matrix(y_compvis_test,y_compvis_predict)[1][0]))
print("The number of true positives for compvis classifier of NN model is: " + str(confusion_matrix(y_compvis_test,y_compvis_predict)[1][1]))

The number of true negatives for Compvis classifier of NN model is: 17464
The number of false positives for Compvis classifier of NN model is: 63
The number of false negatives for compvis classifier of NN model is: 2141
The number of true positives for compvis classifier of NN model is: 11


## Preparing the data for MATH

Here we will be using TabularDataset to read our data and the dataset we are using here is a csv format and then we are usding the specified datafields which we have previously defined.

Then for the train and test data we need to process the labels and the order is same as that of the columns and then the fields which we are not using will pass them to a tuple where the second element is None.

In [0]:
## Applying NN model on Math column
math_data = [("ID", None), # we won't be needing the source and notes, so we pass in None as the field
                 ("URL", None),
                 ("Date", None),
                 ("Title", None),
                 ("InfoTheory", None),
                 ("CompVis", None),
                 ('Math',LABEL),
                 ("Abstract", TEXT)]

train_math, test_math = TabularDataset.splits(path='',
                                              train='axcs_train.csv', 
                                              test='axcs_test.csv', 
                                              format='csv',
                                              skip_header=True,
                                              fields=math_data)

Here Below we have printed the Training examples length and Testing examples length to check how they are splitting.

In [33]:
print(f'Number of training examples: {len(train_math)}')
print(f'Number of testing examples: {len(test_math)}')

Number of training examples: 54731
Number of testing examples: 19679


In [34]:
print(vars(train_math.examples[0]))

{'Math': 0, 'Abstract': [' ', 'nested', 'satisfiability', 'a', 'special', 'case', 'of', 'the', 'satisfiability', 'problem', ',', 'in', 'which', 'the', 'clauses', 'have', 'a', 'hierarchical', 'structure', ',', 'is', 'shown', 'to', 'be', 'solvable', 'in', 'linear', 'time', ',', 'assuming', 'that', 'the', 'clauses', 'have', 'been', 'represented', 'in', 'a', 'convenient', 'way', '.']}


Next we are building the Vocabulary, that is effectively used as a lookup table i,e every word in our data set has a corresonding index we do this operation since machine leaning cannot operate on strings.

So, each index is used to construct a one-hot encoding. in which it trasforms the srings into an inter value of vectors form and then in this we will have more zero's and only one 1's in the vetor which indicate according the index value.

The Dimentionality is the total number of unique words in the vocabulary.

The number of unique words in our Training set is "5400 " and since it would take more memory space we would cut down the training dataset as the top 5000 words.

then we are replacing a special unknown token when there is need for those missing words.

Then we are only keeping the most common max_size tokens.

In [0]:
MAX_VOCAB_SIZE = 5400

TEXT.build_vocab(train_math, max_size = MAX_VOCAB_SIZE)
#LABEL.build_vocab(train_data)

In [36]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
#print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 5402


In [37]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 512010), ('of', 344238), ('.', 332875), (',', 307484), ('a', 218648), ('and', 213963), ('-', 184859), ('to', 176340), ('in', 174585), ('is', 130627), ('for', 120901), ('we', 110661), ('that', 89421), ('this', 74411), ('on', 68779), ('with', 66354), ('are', 57723), (' ', 54731), ('an', 49791), ('by', 49768)]


When we feed the sentence into our model we feed more than one at time and then it should follow a same size as that of the batch size and to ensure each sentence in the batch is the same size, any shorter than the longest within the batch are padded.

In [38]:
print(TEXT.vocab.itos[:10])
print(LABEL)

['<unk>', '<pad>', 'the', 'of', '.', ',', 'a', 'and', '-', 'to']


Here we are initializing the batch size as 16, then we are preparing the data by creating an iterators.

After creating we are iterating over the training and testing and then returning the batch of examples which are converted into tensors by using the indexes at every iterations.

Here below we are using `BucketIterator` which is a special iterator which would return a examples of batches with similar length and minimizing the amount of padding per examples.

Then we are replacing the tensors returned by the iterators on the GPU and this is handled by `torch.device` by passing this device to the iterator.

We also want to place the tensors returned by the iterator on the GPU (if you're using one). PyTorch handles this using `torch.device`, we then pass this device to the iterator.

In [0]:
BATCH_SIZE = 16

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, test_iterator = data.BucketIterator.splits(
    (train_math, test_math), 
    batch_size = BATCH_SIZE,
    device = device,
    sort_key = lambda x: len(x.Abstract),
    sort_within_batch = False)

In [40]:
batch_math = next(train_iterator.__iter__())
batch_math


[torchtext.data.batch.Batch of size 16]
	[.Math]:[torch.cuda.FloatTensor of size 16 (GPU 0)]
	[.Abstract]:[torch.cuda.LongTensor of size 345x16 (GPU 0)]

### Building the model

Here below we are creating some instances for our RNN class and the input dimension here is the one-hot vector which is equal to Vocab size.

Then the embedding dimension is the size of the dense vector which is transformed from sparse vector to dense vector to reduce the size of the dataset so Rnn could learn easily and this depends on the size of the Vocabulary.

The hidden dimentions are nothing but the size hidden states and then this also is depending upon the size of the Vocabulary and also as mentioned above that is from the embedding the dense vector and also dependent in the complecxity of the task

The output dimentions are usually the number of classes. here we are providing with one dimention. which is a single scalar value.

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 260
HIDDEN_DIM = 280
OUTPUT_DIM = 1

model_math = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

Here we are defining a function by passing the above created model and returing the sum of all parameters of the training set.

In [42]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model_math):,} trainable parameters')

The model has 1,556,561 trainable parameters


 # Train the Model for Math

Here we are creating a Optimizer which is an algorithm where we update the parameters of the module.

So, here we are using <b>stochastic gradient descent (SGD)</b> as an optimizer. In this the first argument we are passing is the parameters that has to be updated and then followed by the learning rate.

In [0]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

Here in the below cell have defined a loss function and in PyTorch it is commonly called as Criterion.
This is bacause it is used as criteria for the function to carry out or to terminate.

Here we are using `Binary cross entropy with logits`.

Here we are restricting the predictions to 0 and 1 by suing some sigmoid or logit function.
Then after this we use some bond scalar to calculate the loss by using binary cross entropy and the `BCEWithLogitsLoss` criterion is used to carry out the sigmoid and the binary cross entropy steps.

In [0]:
criterion = nn.BCEWithLogitsLoss()

In [0]:
model = model.to(device)
criterion = criterion.to(device)

- The Train function is made to iterate over all examples i,e. one batch at a time and before that we are training our model and which turns on the dropout and batch normalization.
- First we are putting zeros for all batches and then storing the gradient calculated by the Criterion by making all paramaters in the model to have a grad attribute.
- This is done because PyTorch does not have the capability to remove the gradients calculated i,e. zeros from the gradient calculations, so we are remoiving it manually.

- After the above process we are providing the batch of sentences as batch.text into the model which is done by the forward function as mention and explained above and then it squeeze is needed as the prediction re initially size [batch size, 1], and we need to remove the dimension of size 1 as PyTorch expects the predictions input to our criterion function to be of size [batch size].

- Loss and accuracy are then measured using our forecasts and the package.label labels, with the loss being summed over all the examples in the sample.

- We calculate the gradient of each parameter with loss.backward(), and then update the parameters using the gradients and optimizer algorithm with optimizer.step().

- `.item()` is used to extract a scalar from the tensor in turn which only contains a single value.

- then we are returning the loss and accuracy as an average acorss the number of iterations or steps or epoch and then we are specifying the len of an iterator as the number of batches in the iteartor.



In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.Abstract).squeeze(1)
        
        loss = criterion(predictions, batch.Math)
        
        acc = binary_accuracy(predictions, batch.Math)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Evaluate function is same as the train function but here we are not updating the paramters when evaluating.

model.eval() puts the model in "evaluation mode", this turns off dropout and batch normalization. Again, we are not using them in this model, but it is good practice to include them.

No gradients are calculated on PyTorch operations inside the with no_grad() block. This causes less memory to be used and speeds up computation.

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.Abstract).squeeze(1)
            
            loss = criterion(predictions, batch.Math)
            
            acc = binary_accuracy(predictions, batch.Math)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Here we are training the model through multiple epochs and then as we train model one by one we are passing through all the examples in the training set so the RNN learns it well.

In [50]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_math_loss, train_math_acc = train(model, train_iterator, optimizer, criterion)    
    end_time = time.time()

    epoch_math_mins, epoch_math_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_math_mins}m {epoch_math_secs}s')
    print(f'\tTrain Loss: {train_math_loss:.3f} | Train Acc: {train_math_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 38s
	Train Loss: 0.626 | Train Acc: 69.41%
Epoch: 02 | Epoch Time: 0m 38s
	Train Loss: 0.616 | Train Acc: 69.41%
Epoch: 03 | Epoch Time: 0m 38s
	Train Loss: 0.616 | Train Acc: 69.41%
Epoch: 04 | Epoch Time: 0m 38s
	Train Loss: 0.616 | Train Acc: 69.41%
Epoch: 05 | Epoch Time: 0m 38s
	Train Loss: 0.616 | Train Acc: 69.41%


In [0]:
torch.save(model.state_dict(), 'RNN_model.pt')

In [52]:
model.load_state_dict(torch.load('RNN_model.pt'))


<All keys matched successfully>

In [0]:
y_math_predict = []
y_math_test = []

model.eval()
with torch.no_grad():
    for batch in test_iterator:
        predictions = model(batch.Abstract).squeeze(1)
        rounded_preds = torch.round(torch.sigmoid(predictions))
        y_math_predict += rounded_preds.tolist()
        y_math_test += batch.Math.tolist()
        #acc = binary_accuracy(predictions, batch.Math)

In [54]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, matthews_corrcoef
import numpy as np

y_math_predict = np.asarray(y_math_predict)
y_math_test = np.asarray(y_math_test)
print(confusion_matrix(y_math_test,y_math_predict))
recall_math=recall_score(y_math_test,y_math_predict,average='macro')
precision_math=precision_score(y_math_test,y_math_predict,average='macro')
f1score_math=f1_score(y_math_test,y_math_predict,average='macro')
accuracy_math=accuracy_score(y_math_test,y_math_predict)
matthews_math = matthews_corrcoef(y_math_test,y_math_predict) 
print('Accuracy_Math: '+ str(accuracy_math))
print('Macro Precision_Math: '+ str(precision_math))
print('Macro Recall_Math: '+ str(recall_math))
print('Macro F1 score_Math:'+ str(f1score_math))
print('MCC_Math:'+ str(matthews_math))

[[13690    59]
 [ 5912    18]]
Accuracy_Math: 0.6965801107779866
Macro Precision_Math: 0.4660821782033903
Macro Recall_Math: 0.49937209598686744
Macro F1 score_Math:0.4134789483830936
MCC_Math:-0.009229764118937596


In [55]:
print("The number of true negatives for Math classifier of NN model is: " + str(confusion_matrix(y_math_test,y_math_predict)[0][0]))
print("The number of false positives for Math classifier of NN model is: " + str(confusion_matrix(y_math_test,y_math_predict)[0][1]))
print("The number of false negatives for Math classifier of NN model is: " + str(confusion_matrix(y_math_test,y_math_predict)[1][0]))
print("The number of true positives for Math classifier of NN model is: " + str(confusion_matrix(y_math_test,y_math_predict)[1][1]))

The number of true negatives for Math classifier of NN model is: 13690
The number of false positives for Math classifier of NN model is: 59
The number of false negatives for Math classifier of NN model is: 5912
The number of true positives for Math classifier of NN model is: 18


# Preparing the data for INFOTHEORY

Here we will be using TabularDataset to read our data and the dataset we are using here is a csv format and then we are usding the specified datafields which we have previously defined.

Then for the train and test data we need to process the labels and the order is same as that of the columns and then the fields which we are not using will pass them to a tuple where the second element is None.

In [0]:
## Applying NN model on InfoTheory column
infotheory_data = [("ID", None), # we won't be needing the source and notes, so we pass in None as the field
                 ("URL", None),
                 ("Date", None),
                 ("Title", None),
                 ("InfoTheory", LABEL),
                 ("CompVis", None),
                 ('Math',None),
                 ("Abstract", TEXT)]

train_infotheory, test_infotheory = TabularDataset.splits(path='',
                                              train='axcs_train.csv', 
                                              test='axcs_test.csv', 
                                              format='csv',
                                              skip_header=True,
                                              fields=infotheory_data)

In [57]:
print(f'Number of training examples: {len(train_infotheory)}')
print(f'Number of testing examples: {len(test_infotheory)}')

Number of training examples: 54731
Number of testing examples: 19679


In [58]:
print(vars(train_infotheory.examples[0]))

{'InfoTheory': 0, 'Abstract': [' ', 'nested', 'satisfiability', 'a', 'special', 'case', 'of', 'the', 'satisfiability', 'problem', ',', 'in', 'which', 'the', 'clauses', 'have', 'a', 'hierarchical', 'structure', ',', 'is', 'shown', 'to', 'be', 'solvable', 'in', 'linear', 'time', ',', 'assuming', 'that', 'the', 'clauses', 'have', 'been', 'represented', 'in', 'a', 'convenient', 'way', '.']}


Next we are building the Vocabulary, that is effectively used as a lookup table i,e every word in our data set has a corresonding index we do this operation since machine leaning cannot operate on strings.

So, each index is used to construct a one-hot encoding. in which it trasforms the srings into an inter value of vectors form and then in this we will have more zero's and only one 1's in the vetor which indicate according the index value.

The Dimentionality is the total number of unique words in the vocabulary.

The number of unique words in our Training set is " " and since it would take more memory space we would cut down the training dataset as the top 5000 words.

then we are replacing a special unknown token when there is need for those missing words.

Then we are only keeping the most common max_size tokens.

In [0]:
MAX_VOCAB_SIZE = 5400

TEXT.build_vocab(train_infotheory, max_size = MAX_VOCAB_SIZE)
#LABEL.build_vocab(train_data)

In [60]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
#print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 5402


Printing the frequencies of most common words in the training set.

The vocab size 5002 because of the addition tokens is the `<unk>` token and the other is a `<pad>` token.

In [61]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 512010), ('of', 344238), ('.', 332875), (',', 307484), ('a', 218648), ('and', 213963), ('-', 184859), ('to', 176340), ('in', 174585), ('is', 130627), ('for', 120901), ('we', 110661), ('that', 89421), ('this', 74411), ('on', 68779), ('with', 66354), ('are', 57723), (' ', 54731), ('an', 49791), ('by', 49768)]


When we feed the sentence into our model we feed more than one at time and then it should follow a same size as that of the batch size and to ensure each sentence in the batch is the same size, any shorter than the longest within the batch are padded.

In [62]:
print(TEXT.vocab.itos[:10])
print(LABEL)

['<unk>', '<pad>', 'the', 'of', '.', ',', 'a', 'and', '-', 'to']


Here we are initializing the batch size as 16, then we are preparing the data by creating an iterators.

After creating we are iterating over the training and testing and then returning the batch of examples which are converted into tensors by using the indexes at every iterations.

Here below we are using `BucketIterator` which is a special iterator which would return a examples of batches with similar length and minimizing the amount of padding per examples.

Then we are replacing the tensors returned by the iterators on the GPU and this is handled by `torch.device` by passing this device to the iterator.

We also want to place the tensors returned by the iterator on the GPU (if you're using one). PyTorch handles this using `torch.device`, we then pass this device to the iterator.

In [0]:
BATCH_SIZE = 16

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, test_iterator = data.BucketIterator.splits(
    (train_infotheory, test_infotheory), 
    batch_size = BATCH_SIZE,
    device = device,
    sort_key = lambda x: len(x.Abstract),
    sort_within_batch = False)

In [64]:
batch_infotheory = next(train_iterator.__iter__())
batch_infotheory


[torchtext.data.batch.Batch of size 16]
	[.InfoTheory]:[torch.cuda.FloatTensor of size 16 (GPU 0)]
	[.Abstract]:[torch.cuda.LongTensor of size 345x16 (GPU 0)]

### Building the model

Here below we are creating some instances for our RNN class and the input dimension here is the one-hot vector which is equal to Vocab size.

Then the embedding dimension is the size of the dense vector which is transformed from sparse vector to dense vector to reduce the size of the dataset so Rnn could learn easily and this depends on the size of the Vocabulary.

The hidden dimentions are nothing but the size hidden states and then this also is depending upon the size of the Vocabulary and also as mentioned above that is from the embedding the dense vector and also dependent in the complecxity of the task

The output dimentions are usually the number of classes. here we are providing with one dimention. which is a single scalar value.

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 255
HIDDEN_DIM = 236
OUTPUT_DIM = 1

model_infotheory = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

Here we are defining a function by passing the above created model and returing the sum of all parameters of the training set.

In [66]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model_infotheory):,} trainable parameters')

The model has 1,494,095 trainable parameters


 # Train the Model for InfoTheory

Here we are creating a Optimizer which is an algorithm where we update the parameters of the module.

So, here we are using <b>stochastic gradient descent (SGD)</b> as an optimizer. In this the first argument we are passing is the parameters that has to be updated and then followed by the learning rate.


In [0]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

Here in the below cell have defined a loss function and in PyTorch it is commonly called as Criterion.
This is bacause it is used as criteria for the function to carry out or to terminate.

Here we are using `Binary cross entropy with logits`.

Here we are restricting the predictions to 0 and 1 by suing some sigmoid or logit function.
Then after this we use some bond scalar to calculate the loss by using binary cross entropy and the `BCEWithLogitsLoss` criterion is used to carry out the sigmoid and the binary cross entropy steps.

In [0]:
criterion = nn.BCEWithLogitsLoss()

In [0]:
model = model.to(device)
criterion = criterion.to(device)

- The Train function is made to iterate over all examples i,e. one batch at a time and before that we are training our model and which turns on the dropout and batch normalization.
- First we are putting zeros for all batches and then storing the gradient calculated by the Criterion by making all paramaters in the model to have a grad attribute.
- This is done because PyTorch does not have the capability to remove the gradients calculated i,e. zeros from the gradient calculations, so we are remoiving it manually.

- After the above process we are providing the batch of sentences as batch.text into the model which is done by the forward function as mention and explained above and then it squeeze is needed as the prediction re initially size [batch size, 1], and we need to remove the dimension of size 1 as PyTorch expects the predictions input to our criterion function to be of size [batch size].

- Loss and accuracy are then measured using our forecasts and the package.label labels, with the loss being summed over all the examples in the sample.

- We calculate the gradient of each parameter with loss.backward(), and then update the parameters using the gradients and optimizer algorithm with optimizer.step().

- `.item()` is used to extract a scalar from the tensor in turn which only contains a single value.

- then we are returning the loss and accuracy as an average acorss the number of iterations or steps or epoch and then we are specifying the len of an iterator as the number of batches in the iteartor.



In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.Abstract).squeeze(1)
        
        loss = criterion(predictions, batch.InfoTheory)
        
        acc = binary_accuracy(predictions, batch.InfoTheory)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Evaluate function is same as the train function but here we are not updating the paramters when evaluating.

model.eval() puts the model in "evaluation mode", this turns off dropout and batch normalization. Again, we are not using them in this model, but it is good practice to include them.

No gradients are calculated on PyTorch operations inside the with no_grad() block. This causes less memory to be used and speeds up computation.

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.Abstract).squeeze(1)
            
            loss = criterion(predictions, batch.InfoTheory)
            
            acc = binary_accuracy(predictions, batch.InfoTheory)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Here we are training the model through multiple epochs and then as we train model one by one we are passing through all the examples in the training set so the RNN learns it well.

In [74]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_infotheory_loss, train_infotheory_acc = train(model, train_iterator, optimizer, criterion)    
    end_time = time.time()

    epoch_infotheory_mins, epoch_infotheory_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_infotheory_mins}m {epoch_infotheory_secs}s')
    print(f'\tTrain Loss: {train_infotheory_loss:.3f} | Train Acc: {train_infotheory_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 38s
	Train Loss: 0.490 | Train Acc: 80.72%
Epoch: 02 | Epoch Time: 0m 38s
	Train Loss: 0.490 | Train Acc: 80.73%
Epoch: 03 | Epoch Time: 0m 38s
	Train Loss: 0.490 | Train Acc: 80.72%
Epoch: 04 | Epoch Time: 0m 38s
	Train Loss: 0.490 | Train Acc: 80.72%
Epoch: 05 | Epoch Time: 0m 38s
	Train Loss: 0.490 | Train Acc: 80.72%


In [0]:
torch.save(model.state_dict(), 'RNN_model.pt')

In [76]:
model.load_state_dict(torch.load('RNN_model.pt'))

<All keys matched successfully>

In [0]:
y_infotheory_predict = []
y_infotheory_test = []

model.eval()
with torch.no_grad():
    for batch in test_iterator:
        predictions = model(batch.Abstract).squeeze(1)
        rounded_preds = torch.round(torch.sigmoid(predictions))
        y_infotheory_predict += rounded_preds.tolist()
        y_infotheory_test += batch.InfoTheory.tolist()
        #acc = binary_accuracy(predictions, batch.InfoTheory)

In [78]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, matthews_corrcoef
import numpy as np

y_infotheory_predict = np.asarray(y_infotheory_predict)
y_infotheory_test = np.asarray(y_infotheory_test)
print(confusion_matrix(y_infotheory_test,y_infotheory_predict))
recall_Infotheory=recall_score(y_infotheory_test,y_infotheory_predict,average='macro')
precision_Infotheory=precision_score(y_infotheory_test,y_infotheory_predict,average='macro')
f1score_Infotheory=f1_score(y_infotheory_test,y_infotheory_predict,average='macro')
accuracy_Infotheory=accuracy_score(y_infotheory_test,y_infotheory_predict)
matthews_Infotheory = matthews_corrcoef(y_infotheory_test,y_infotheory_predict) 
print('Accuracy_Infotheory: '+ str(accuracy_Infotheory))
print('Macro Precision_Infotheory: '+ str(precision_Infotheory))
print('Macro Recall_Infotheory: '+ str(recall_Infotheory))
print('Macro F1 score_Infotheory:'+ str(f1score_Infotheory))
print('MCC_Infotheory:'+ str(matthews_Infotheory))

[[16008    55]
 [ 3609     7]]
Accuracy_Infotheory: 0.8138116774226333
Macro Precision_Infotheory: 0.4644650706184728
Macro Recall_Infotheory: 0.4992559113892808
Macro F1 score_Infotheory:0.450557916785541
MCC_Infotheory:-0.010284189075567409


In [79]:
print("The number of true negatives for Infotheory classifier of NN model is: " + str(confusion_matrix(y_infotheory_test,y_infotheory_predict)[0][0]))
print("The number of false positives for Infotheory classifier of NN model is: " + str(confusion_matrix(y_infotheory_test,y_infotheory_predict)[0][1]))
print("The number of false negatives for Infotheory classifier of NN model is: " + str(confusion_matrix(y_infotheory_test,y_infotheory_predict)[1][0]))
print("The number of true positives for Infotheory classifier of NN model is: " + str(confusion_matrix(y_infotheory_test,y_infotheory_predict)[1][1]))

The number of true negatives for Infotheory classifier of NN model is: 16008
The number of false positives for Infotheory classifier of NN model is: 55
The number of false negatives for Infotheory classifier of NN model is: 3609
The number of true positives for Infotheory classifier of NN model is: 7


### Part 1: Machine Learning Method

In [22]:
from nltk.corpus import stopwords
from nltk import word_tokenize    
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB,BernoulliNB
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import text
#from sklearn import cross_validation
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
import re

# Math

In [23]:
data_training = pd.read_csv("axcs_train.csv")
data_testing = pd.read_csv("axcs_test.csv")

In [24]:
data_training.head()

,ID,URL,Date,Title,InfoTheory,CompVis,Math,Abstract
0,cs-9301111,arxiv.org/abs/cs/9301111,1989-12-31,Nested satisfiability,0,0,0,Nested satisfiability A special case of the s...
1,cs-9301112,arxiv.org/abs/cs/9301112,1990-03-31,A note on digitized angles,0,0,0,A note on digitized angles We study the confi...
2,cs-9301113,arxiv.org/abs/cs/9301113,1991-07-31,Textbook examples of recursion,0,0,0,Textbook examples of recursion We discuss pro...
3,cs-9301114,arxiv.org/abs/cs/9301114,1991-10-31,Theory and practice,0,0,0,Theory and practice The author argues to Sili...
4,cs-9301115,arxiv.org/abs/cs/9301115,1991-11-30,Context-free multilanguages,0,0,0,Context-free multilanguages This article is a...


In [25]:
data_testing.head() 

,ID,URL,Date,Title,InfoTheory,CompVis,Math,Abstract
0,no-150100335,arxiv.org/abs/1501.00335,1/1/15,A Data Transparency Framework for Mobile Appli...,0,0,0,A Data Transparency Framework for Mobile Appl...
1,no-14024178,arxiv.org/abs/1402.4178,1/1/15,A reclaimer scheduling problem arising in coal...,0,0,0,A reclaimer scheduling problem arising in coa...
2,no-150100263,arxiv.org/abs/1501.00263,1/1/15,Communication-Efficient Distributed Optimizati...,0,0,1,Communication-Efficient Distributed Optimizat...
3,no-150100287,arxiv.org/abs/1501.00287,1/1/15,Consistent Classification Algorithms for Multi...,0,0,0,Consistent Classification Algorithms for Mult...
4,no-11070586,arxiv.org/abs/1107.0586,1/1/15,Managing key multicasting through orthogonal s...,0,0,0,Managing key multicasting through orthogonal ...


In [26]:
Abstract_training_data = data_training.Abstract.tolist()   # convert into a list
Math_training_data = data_training.Math.tolist() 

In [27]:
Abstract_training_data[0:5]

[' Nested satisfiability A special case of the satisfiability problem, in which the clauses have a hierarchical structure, is shown to be solvable in linear time, assuming that the clauses have been represented in a convenient way. ',
 ' A note on digitized angles We study the configurations of pixels that occur when two digitized straight lines meet each other. ',
 " Textbook examples of recursion We discuss properties of recursive schemas related to McCarthy's ``91 function'' and to Takeuchi's triple recursion. Several theorems are proposed as interesting candidates for machine verification, and some intriguing open questions are raised. ",
 ' Theory and practice The author argues to Silicon Valley that the most important and powerful part of computer science is work that is simultaneously theoretical and practical. He particularly considers the intersection of the theory of algorithms and practical software development. He combines examples from the development of the TeX typesettin

In [28]:
Math_training_data[0:5]

[0, 0, 0, 0, 0]

In [29]:
print(len(Abstract_training_data)) 

54731


In [30]:
class LemmaTokenizer(object):                            # creating a class for lemmatokenizer
    def __init__(self):
        self.wnl=WordNetLemmatizer()
    def __call__(self,doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [31]:
# Tokenizing the data and removing stop words
stop_words_for_vec = text.ENGLISH_STOP_WORDS                           
vectorizer = TfidfVectorizer(analyzer='word',input='content',
                           lowercase=True,
                           token_pattern=r"[A-Za-z]\w+(?:[-'?]\w+)?", stop_words = set(stop_words_for_vec),
                           min_df=3,
                           ngram_range=(1,2),
                           tokenizer=LemmaTokenizer())

In [32]:
# Training the data as x_train and y_train
x_train = vectorizer.fit_transform(Abstract_training_data)
y_train = np.asarray(Math_training_data)

/Users/manojmanu/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [33]:
# building models like  LogisticRegression, BernoulliNB,LinearSVC,RandomForestClassifier and doing some cross validation.
models = [
    LogisticRegression(),
    BernoulliNB(),
    LinearSVC(),
    RandomForestClassifier()
]
CV = 10
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
     model_name = model.__class__.__name__
     accuracies = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=CV)
     for fold_idx, accuracy in enumerate(accuracies):
          entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

/Users/manojmanu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/manojmanu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/manojmanu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/manojmanu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/manojmanu/anaconda3/lib/python3.7/site-package

In [34]:
# converting to a list
Abstract_testing_data = data_testing.Abstract.tolist()    
Math_testing_data = data_testing.Math.tolist()

In [35]:
x_test=vectorizer.transform(Abstract_testing_data)
y_test=np.asarray(Math_testing_data)

In [36]:
# Finding the Accuracy, Macro precision, Macro Recall, Macro F1 score, MCC
models = [
    LogisticRegression(),
    BernoulliNB(),
    LinearSVC(),
    RandomForestClassifier()
]

for clf in models:
    model_name = clf.__class__.__name__
    clf.fit(x_train, y_train)
    print(model_name)
    # Do the prediction
    y_predict=clf.predict(x_test)
    print(confusion_matrix(y_test,y_predict))
    recall=recall_score(y_test,y_predict,average='macro')
    precision=precision_score(y_test,y_predict,average='macro')
    f1score=f1_score(y_test,y_predict,average='macro')
    accuracy=accuracy_score(y_test,y_predict)
    matthews = matthews_corrcoef(y_test,y_predict) 
    print('Accuracy: '+ str(accuracy))
    print('Macro Precision: '+ str(precision))
    print('Macro Recall: '+ str(recall))
    print('Macro F1 score:'+ str(f1score))
    print('MCC:'+ str(matthews))

/Users/manojmanu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression
[[13020   729]
 [ 1702  4228]]
Accuracy: 0.8764673001676915
Macro Precision: 0.8686629754374238
Macro Recall: 0.8299813924839176
Macro F1 score:0.8456605172932836
MCC:0.697572711599602
BernoulliNB
[[12519  1230]
 [ 1306  4624]]
Accuracy: 0.871131663194268
Macro Precision: 0.847710355232292
Macro Recall: 0.8451514302987174
Macro F1 score:0.8464111212728265
MCC:0.6928570601158675
LinearSVC
[[12814   935]
 [ 1377  4553]]
Accuracy: 0.8825143554042381
Macro Precision: 0.8662974744494776
Macro Recall: 0.8498929739731493
Macro F1 score:0.8573819759655634
MCC:0.7160025494199902


/Users/manojmanu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier
[[13382   367]
 [ 3277  2653]]
Accuracy: 0.8148279892270949
Macro Precision: 0.840883167184057
Macro Recall: 0.7103466608088131
Macro F1 score:0.736506138490459
MCC:0.5355506920660219


# INFOTHEORY

In [37]:
Abstract_training_data = data_training.Abstract.tolist()
InfoTheory_training_data = data_training.InfoTheory.tolist()

In [38]:
Abstract_training_data[0:5]

[' Nested satisfiability A special case of the satisfiability problem, in which the clauses have a hierarchical structure, is shown to be solvable in linear time, assuming that the clauses have been represented in a convenient way. ',
 ' A note on digitized angles We study the configurations of pixels that occur when two digitized straight lines meet each other. ',
 " Textbook examples of recursion We discuss properties of recursive schemas related to McCarthy's ``91 function'' and to Takeuchi's triple recursion. Several theorems are proposed as interesting candidates for machine verification, and some intriguing open questions are raised. ",
 ' Theory and practice The author argues to Silicon Valley that the most important and powerful part of computer science is work that is simultaneously theoretical and practical. He particularly considers the intersection of the theory of algorithms and practical software development. He combines examples from the development of the TeX typesettin

In [39]:
InfoTheory_training_data[0:5]

[0, 0, 0, 0, 0]

In [40]:
x_train = vectorizer.fit_transform(Abstract_training_data)
y_train = np.asarray(InfoTheory_training_data)

In [41]:
len(vectorizer.get_feature_names())

286170

In [42]:
models = [
    LogisticRegression(),
    BernoulliNB(),
    LinearSVC(),
    RandomForestClassifier()
]
CV = 10
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
     model_name = model.__class__.__name__
     accuracies = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=CV)
     for fold_idx, accuracy in enumerate(accuracies):
          entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

/Users/manojmanu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/manojmanu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/manojmanu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/manojmanu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/manojmanu/anaconda3/lib/python3.7/site-package

In [43]:
Abstract_testing_data = data_testing.Abstract.tolist()
InfoTheory_testing_data = data_testing.InfoTheory.tolist()

In [44]:
x_test=vectorizer.transform(Abstract_testing_data)
y_test=np.asarray(InfoTheory_testing_data)

In [45]:
models = [
    LogisticRegression(),
    BernoulliNB(),
    LinearSVC(),
    RandomForestClassifier()
]

for clf in models:
    model_name = clf.__class__.__name__
    clf.fit(x_train, y_train)
    print(model_name)
    # Do the prediction
    y_predict=clf.predict(x_test)
    print(confusion_matrix(y_test,y_predict))
    recall=recall_score(y_test,y_predict,average='macro')
    precision=precision_score(y_test,y_predict,average='macro')
    f1score=f1_score(y_test,y_predict,average='macro')
    accuracy=accuracy_score(y_test,y_predict)
    matthews = matthews_corrcoef(y_test,y_predict) 
    print('Accuracy: '+ str(accuracy))
    print('Macro Precision: '+ str(precision))
    print('Macro Recall: '+ str(recall))
    print('Macro F1 score:'+ str(f1score))
    print('MCC:'+ str(matthews))

/Users/manojmanu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression
[[15881   182]
 [  851  2765]]
Accuracy: 0.9475074952995579
Macro Precision: 0.9436908269701535
Macro Recall: 0.876663346521633
Macro F1 score:0.9055518821563093
MCC:0.8176113299301296
BernoulliNB
[[15658   405]
 [  594  3022]]
Accuracy: 0.9492352253671427
Macro Precision: 0.9226357433882932
Macro Recall: 0.9052584327804403
Macro F1 score:0.9136212996669142
MCC:0.8277117831770574
LinearSVC
[[15818   245]
 [  598  3018]]
Accuracy: 0.9571624574419432
Macro Precision: 0.9442439232424029
Macro Recall: 0.9096857251508028
Macro F1 score:0.9257489427869707
MCC:0.8532300834766704


/Users/manojmanu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier
[[15888   175]
 [ 1388  2228]]
Accuracy: 0.9205752324813252
Macro Precision: 0.9234158467309088
Macro Recall: 0.8026279199876152
Macro F1 score:0.8467201413108167
MCC:0.7159258536635474


# Compvis

In [46]:
Abstract_training_data = data_training.Abstract.tolist()
Compvis_training_data = data_training.CompVis.tolist()

In [47]:
x_train = vectorizer.fit_transform(Abstract_training_data)
y_train = np.asarray(Compvis_training_data)

In [48]:
models = [
    LogisticRegression(),
    BernoulliNB(),
    LinearSVC(),
    RandomForestClassifier()
]
CV = 10
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
     model_name = model.__class__.__name__
     accuracies = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=CV)
     for fold_idx, accuracy in enumerate(accuracies):
          entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

/Users/manojmanu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/manojmanu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/manojmanu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/manojmanu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/manojmanu/anaconda3/lib/python3.7/site-package

In [49]:
Abstract_testing_data = data_testing.Abstract.tolist()
Compvis_testing_data = data_testing.CompVis.tolist()

In [50]:
x_test=vectorizer.transform(Abstract_testing_data)
y_test=np.asarray(Compvis_testing_data)

In [51]:
models = [
    LogisticRegression(),
    BernoulliNB(),
    LinearSVC(),
    RandomForestClassifier()
]

for clf in models:
    model_name = clf.__class__.__name__
    clf.fit(x_train, y_train)
    print(model_name)
    # Do the prediction
    y_predict=clf.predict(x_test)
    print(confusion_matrix(y_test,y_predict))
    recall=recall_score(y_test,y_predict,average='macro')
    precision=precision_score(y_test,y_predict,average='macro')
    f1score=f1_score(y_test,y_predict,average='macro')
    accuracy=accuracy_score(y_test,y_predict)
    matthews = matthews_corrcoef(y_test,y_predict) 
    print('Accuracy: '+ str(accuracy))
    print('Macro Precision: '+ str(precision))
    print('Macro Recall: '+ str(recall))
    print('Macro F1 score:'+ str(f1score))
    print('MCC:'+ str(matthews))

/Users/manojmanu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression
[[17462    65]
 [  833  1319]]
Accuracy: 0.9543675999796738
Macro Precision: 0.9537515580396425
Macro Recall: 0.8046048258417231
Macro F1 score:0.8604861651286868
MCC:0.7435453296526738
BernoulliNB
[[17477    50]
 [ 1299   853]]
Accuracy: 0.9314497687890645
Macro Precision: 0.9377224748164642
Macro Recall: 0.6967613615997241
Macro F1 score:0.7606346709160439
MCC:0.5869475961197504
LinearSVC
[[17436    91]
 [  472  1680]]
Accuracy: 0.9713908227044057
Macro Precision: 0.9611298326713347
Macro Recall: 0.8877385777397506
Macro F1 score:0.9202995680105757
MCC:0.8456898378799464


/Users/manojmanu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier
[[17494    33]
 [ 1486   666]]
Accuracy: 0.9228111184511408
Macro Precision: 0.9372483798170199
Macro Recall: 0.6537983722617658
Macro F1 score:0.712798015500494
MCC:0.5186447303885465


## Part 2: Topic Modelling

In [52]:
import pandas as pd
from functools import reduce
from nltk.corpus import stopwords
import nltk

In [53]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/manojmanu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [54]:
data = pd.read_csv('Monash_crawled.csv')
docs = data['body'].tolist()
print(len(docs))
print(docs[0][0:500])

366
 Canberra
 has experienced its worst air quality on record, as bushfire smoke 
became trapped by atmospheric conditions and residents were told to stay
 indoors and brace for more smog in the coming days.   The ACTas 
acting chief health officer, Dr Paul Dugdale, said the smoke was the 
worst since the 2003 bushfires and was acertainly the worsta since air 
quality monitoring started in the city 15 years ago.   Air quality 
index readings in Canberra city were at 3,463 on Wednesday afternoon, 
a


In [55]:
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

In [56]:
# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

In [57]:
# !pip install gensim
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [58]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.6)

In [59]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [60]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 1441
Number of documents: 366


In [61]:
# Train LDA model.
import numpy as np
seed_value= 1234
np.random.seed(seed_value)
from gensim.models import LdaModel

# Set training parameters.
NUM_TOPICS = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=NUM_TOPICS,
    passes=passes,
    eval_every=eval_every
)
print(model)
outputfile = f'model{NUM_TOPICS}.gensim'
print("Saving model in " + outputfile)
print("")
model.save(outputfile)

LdaModel(num_terms=1441, num_topics=10, decay=0.5, chunksize=2000)
Saving model in model10.gensim



In [62]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / NUM_TOPICS
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -0.9243.
[([(0.023180436, 'student'),
   (0.016972676, 'china'),
   (0.014348613, 'chinese'),
   (0.009738987, 'flight'),
   (0.009387481, 'sydney'),
   (0.00938562, 'travel'),
   (0.009093791, 'virus'),
   (0.008612675, 'week'),
   (0.008259719, 'ban'),
   (0.008217118, 'melbourne'),
   (0.0074494113, 'february'),
   (0.0070628272, 'she'),
   (0.006682953, 'country'),
   (0.006502782, 'wuhan'),
   (0.0062591773, 'two'),
   (0.006152358, 'confirmed'),
   (0.0058028502, 'case'),
   (0.0055985427, 'travel_ban'),
   (0.0054341424, 'queensland'),
   (0.0053102737, 'infected')],
  -0.5773085022167089),
 ([(0.03931186, 'february'),
   (0.0122434925, 'ship'),
   (0.0112485085, 'passenger'),
   (0.009737816, 'japan'),
   (0.009051105, 'cruise'),
   (0.008047696, 'february_february'),
   (0.008007615, 'case'),
   (0.007935638, 'flight'),
   (0.0077038086, 'virus'),
   (0.0075195804, 'january_january'),
   (0.00691677, 'quarantine'),
   (0.006861771, 'wuhan'),
   (0.0068

In [63]:
# !pip install pyLDAvis
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)

/Users/manojmanu/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [64]:
import numpy as np
from gensim.models import LdaModel
seed_value= 1234
np.random.seed(seed_value)
# Set training parameters.
NUM_TOPICS = 30
chunksize = 2000
passes = 30
iterations = 500
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=NUM_TOPICS,
    passes=passes,
    eval_every=eval_every
)
print(model)
outputfile = f'model{NUM_TOPICS}.gensim'
print("Saving model in " + outputfile)
print("")
model.save(outputfile)

LdaModel(num_terms=1441, num_topics=30, decay=0.5, chunksize=2000)
Saving model in model30.gensim



In [65]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / NUM_TOPICS
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -1.0024.
[([(0.059220396, 'february'),
   (0.02050294, 'wuhan'),
   (0.01890853, 'island'),
   (0.016768293, 'flight'),
   (0.014672752, 'christmas'),
   (0.014336443, 'christmas_island'),
   (0.012821459, 'case'),
   (0.012744003, 'february_february'),
   (0.012593099, 'china'),
   (0.0122646885, 'january_january'),
   (0.0117761055, 'confirmed'),
   (0.011633937, 'virus'),
   (0.010158211, 'queensland'),
   (0.009733864, 'evacuee'),
   (0.009469852, 'aedt'),
   (0.009134288, 'two'),
   (0.007892864, 'quarantine'),
   (0.007821722, 'western'),
   (0.007560936, 'south_australia'),
   (0.00698679, 'japan')],
  -0.4287685850205632),
 ([(0.03412316, 'ship'),
   (0.024325259, 'cruise'),
   (0.021915182, 'passenger'),
   (0.019432295, 'february'),
   (0.018243931, 'princess'),
   (0.016628819, 'diamond'),
   (0.016510328, 'diamond_princess'),
   (0.016370088, 'cruise_ship'),
   (0.014741474, 'japan'),
   (0.011055808, 'board'),
   (0.010524059, 'on_board'),
   (0.00

In [66]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)

/Users/manojmanu/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
